In [1]:
import pandas as pnds
import numpy as nmpy
import nltk
import re
import matplotlib.pyplot as matplt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords



ModuleNotFoundError: No module named 'pandas'

In [2]:
# pandas kütüphanesi yardımıyla veri setlerimizi yüklüyoruz
dataFrame = pnds.read_csv('NLPlabeledData.tsv',  delimiter="\t", quoting=3)



NameError: name 'pnds' is not defined

In [504]:
#veri setimizin uzunluğu
len(dataFrame["review"])

25000

In [505]:
# ilk n verimiz
dataFrame.head(10)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."
5,"""8196_8""",1,"""I dont know why people think this is such a b..."
6,"""7166_2""",0,"""This movie could have been very good, but com..."
7,"""10633_1""",0,"""I watched this video at a friend's house. I'm..."
8,"""319_1""",0,"""A friend of mine bought this film for £1, and..."
9,"""8713_10""",1,"""<br /><br />This movie is full of references...."


In [506]:
dataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


## Fazlalıkları temizleme

In [507]:
# n. veri setimiz
example_review= dataFrame.review[17]
example_review

'"Ulli Lommel\'s 1980 film \'The Boogey Man\' is no classic, but it\'s an above average low budget chiller that\'s worth a look. The sequel, 1983s \'Boogey Man II\' is ultimately a waste of time, but at the very least it\'s an entertaining one if not taken the least bit seriously. Now II left the door open for another sequel, and I for one wouldn\'t have minded seeing at least one more. One day while I was browsing though the videos at a store in the mall I came across a film entitled \'Return of the Boogey Man.\' When I found out it was a sequel to the earlier films I was happy to shell out a few bucks for it...I should have known better. Though the opening title is \'Boogey Man 3,\' this is no sequel to those two far superior films I named above. Well, not totally anyway.<br /><br />Pros: Ha! That\'s a laugh. Is there anything good about this hunk of cow dung? Let\'s see...it has footage from \'The Boogey Man\' and, um...it\'s mercifully short. Yeah, that\'s about it.<br /><br />Cons

In [508]:
# küçük harfe dönüştürüyoruz(algılama karmaşıklığını önlemek için)
example_review = example_review.lower()
example_review

'"ulli lommel\'s 1980 film \'the boogey man\' is no classic, but it\'s an above average low budget chiller that\'s worth a look. the sequel, 1983s \'boogey man ii\' is ultimately a waste of time, but at the very least it\'s an entertaining one if not taken the least bit seriously. now ii left the door open for another sequel, and i for one wouldn\'t have minded seeing at least one more. one day while i was browsing though the videos at a store in the mall i came across a film entitled \'return of the boogey man.\' when i found out it was a sequel to the earlier films i was happy to shell out a few bucks for it...i should have known better. though the opening title is \'boogey man 3,\' this is no sequel to those two far superior films i named above. well, not totally anyway.<br /><br />pros: ha! that\'s a laugh. is there anything good about this hunk of cow dung? let\'s see...it has footage from \'the boogey man\' and, um...it\'s mercifully short. yeah, that\'s about it.<br /><br />cons

In [509]:
# BeatifulSoup modülü yardımıyla html taglerini temizliyoruz
example_review = BeautifulSoup(example_review).get_text()
example_review

'"ulli lommel\'s 1980 film \'the boogey man\' is no classic, but it\'s an above average low budget chiller that\'s worth a look. the sequel, 1983s \'boogey man ii\' is ultimately a waste of time, but at the very least it\'s an entertaining one if not taken the least bit seriously. now ii left the door open for another sequel, and i for one wouldn\'t have minded seeing at least one more. one day while i was browsing though the videos at a store in the mall i came across a film entitled \'return of the boogey man.\' when i found out it was a sequel to the earlier films i was happy to shell out a few bucks for it...i should have known better. though the opening title is \'boogey man 3,\' this is no sequel to those two far superior films i named above. well, not totally anyway.pros: ha! that\'s a laugh. is there anything good about this hunk of cow dung? let\'s see...it has footage from \'the boogey man\' and, um...it\'s mercifully short. yeah, that\'s about it.cons: where to start? decisi

In [510]:
# regex modülü yardımıyla harf olmayan karakterleri temizliyoruz
example_review = re.sub("[^A-Za-z]",' ',example_review)
example_review

' ulli lommel s      film  the boogey man  is no classic  but it s an above average low budget chiller that s worth a look  the sequel      s  boogey man ii  is ultimately a waste of time  but at the very least it s an entertaining one if not taken the least bit seriously  now ii left the door open for another sequel  and i for one wouldn t have minded seeing at least one more  one day while i was browsing though the videos at a store in the mall i came across a film entitled  return of the boogey man   when i found out it was a sequel to the earlier films i was happy to shell out a few bucks for it   i should have known better  though the opening title is  boogey man     this is no sequel to those two far superior films i named above  well  not totally anyway pros  ha  that s a laugh  is there anything good about this hunk of cow dung  let s see   it has footage from  the boogey man  and  um   it s mercifully short  yeah  that s about it cons  where to start  decisions  decisions  fir

In [511]:
# nltk vasıtasıyla stopwords indiriyoruz 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/birkanardamalkoc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [512]:
# stopwords lerden kurtulmak için split ile kelimeleri bölüyoruz ve listeye dönüştürüyoruz
example_review = example_review.split()
example_review

['ulli',
 'lommel',
 's',
 'film',
 'the',
 'boogey',
 'man',
 'is',
 'no',
 'classic',
 'but',
 'it',
 's',
 'an',
 'above',
 'average',
 'low',
 'budget',
 'chiller',
 'that',
 's',
 'worth',
 'a',
 'look',
 'the',
 'sequel',
 's',
 'boogey',
 'man',
 'ii',
 'is',
 'ultimately',
 'a',
 'waste',
 'of',
 'time',
 'but',
 'at',
 'the',
 'very',
 'least',
 'it',
 's',
 'an',
 'entertaining',
 'one',
 'if',
 'not',
 'taken',
 'the',
 'least',
 'bit',
 'seriously',
 'now',
 'ii',
 'left',
 'the',
 'door',
 'open',
 'for',
 'another',
 'sequel',
 'and',
 'i',
 'for',
 'one',
 'wouldn',
 't',
 'have',
 'minded',
 'seeing',
 'at',
 'least',
 'one',
 'more',
 'one',
 'day',
 'while',
 'i',
 'was',
 'browsing',
 'though',
 'the',
 'videos',
 'at',
 'a',
 'store',
 'in',
 'the',
 'mall',
 'i',
 'came',
 'across',
 'a',
 'film',
 'entitled',
 'return',
 'of',
 'the',
 'boogey',
 'man',
 'when',
 'i',
 'found',
 'out',
 'it',
 'was',
 'a',
 'sequel',
 'to',
 'the',
 'earlier',
 'films',
 'i',
 'wa

In [513]:
len(example_review)

463

In [514]:
# örnek verimiz stopwords lerden arındıktan sonra
stop_words = set(stopwords.words("english"))                      # conversion into set for fast searching
example_review = [w for w in example_review if w not in stop_words]               
example_review


['ulli',
 'lommel',
 'film',
 'boogey',
 'man',
 'classic',
 'average',
 'low',
 'budget',
 'chiller',
 'worth',
 'look',
 'sequel',
 'boogey',
 'man',
 'ii',
 'ultimately',
 'waste',
 'time',
 'least',
 'entertaining',
 'one',
 'taken',
 'least',
 'bit',
 'seriously',
 'ii',
 'left',
 'door',
 'open',
 'another',
 'sequel',
 'one',
 'minded',
 'seeing',
 'least',
 'one',
 'one',
 'day',
 'browsing',
 'though',
 'videos',
 'store',
 'mall',
 'came',
 'across',
 'film',
 'entitled',
 'return',
 'boogey',
 'man',
 'found',
 'sequel',
 'earlier',
 'films',
 'happy',
 'shell',
 'bucks',
 'known',
 'better',
 'though',
 'opening',
 'title',
 'boogey',
 'man',
 'sequel',
 'two',
 'far',
 'superior',
 'films',
 'named',
 'well',
 'totally',
 'anyway',
 'pros',
 'ha',
 'laugh',
 'anything',
 'good',
 'hunk',
 'cow',
 'dung',
 'let',
 'see',
 'footage',
 'boogey',
 'man',
 'um',
 'mercifully',
 'short',
 'yeah',
 'cons',
 'start',
 'decisions',
 'decisions',
 'first',
 'movie',
 'total',
 'bore

In [515]:
len(example_review)

224

In [516]:
#şimdi bu gerçeklediğimiz işlemi bütün veriler için yapacağız

In [517]:
def process(review):
    # html taglerinden arınıyoruz
    review = BeautifulSoup(review).get_text()
    # sayılar,noktalama işaretleri..
    review = re.sub("[^A-Za-z]",' ',review)
    # küçük harf, listeleme
    review = review.lower()
    review = review.split()
    # stopwords
    stop_words = set(stopwords.words("english"))                      
    review = [w for w in review if w not in stop_words]               
    # parçalanmış paragrafları boşluklarından(" ") birleştiriyoruz
    return(" ".join(review))

In [518]:
# training datamızı yukardaki fonksiyon yardımıyla temizliyoruz: 
# her 1000 review sonrası bir satır yazdırarak review işleminin durumunu görüyoruz..

train_x_tum = []
for r in range(len(dataFrame["review"])):        
    if (r+1)%500 == 0:        
        print("Taranan işlem sayısı =", r+1)
    train_x_tum.append(process(dataFrame["review"][r]))

/Users/birkanardamalkoc/opt/anaconda3/envs/tf/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


Taranan işlem sayısı = 500
Taranan işlem sayısı = 1000
Taranan işlem sayısı = 1500
Taranan işlem sayısı = 2000
Taranan işlem sayısı = 2500
Taranan işlem sayısı = 3000
Taranan işlem sayısı = 3500
Taranan işlem sayısı = 4000
Taranan işlem sayısı = 4500
Taranan işlem sayısı = 5000
Taranan işlem sayısı = 5500
Taranan işlem sayısı = 6000
Taranan işlem sayısı = 6500
Taranan işlem sayısı = 7000
Taranan işlem sayısı = 7500
Taranan işlem sayısı = 8000
Taranan işlem sayısı = 8500
Taranan işlem sayısı = 9000
Taranan işlem sayısı = 9500
Taranan işlem sayısı = 10000
Taranan işlem sayısı = 10500
Taranan işlem sayısı = 11000
Taranan işlem sayısı = 11500
Taranan işlem sayısı = 12000
Taranan işlem sayısı = 12500
Taranan işlem sayısı = 13000
Taranan işlem sayısı = 13500
Taranan işlem sayısı = 14000
Taranan işlem sayısı = 14500
Taranan işlem sayısı = 15000
Taranan işlem sayısı = 15500
Taranan işlem sayısı = 16000
Taranan işlem sayısı = 16500
Taranan işlem sayısı = 17000
Taranan işlem sayısı = 17500
Taran

### Train, test split...

In [519]:
x = train_x_tum
y = nmpy.array(dataFrame["sentiment"])

# train test split
train_x, test_x, y_train, y_test = train_test_split(x,y, test_size = 0.1)


### Bag of Words

Verilerimizi temizledik şimdi yapay zekanın çalışması için bu metin bag of words matrisine çeviriyoruz. İşte bu amaçla sklearn içinde bulunan CountVectorizer modülünü kullanıyoruz.

In [520]:
# sklearn içinde bulunan countvectorizer fonksiyonunu kullanarak max 8500 kelimelik bag of words oluşturuyoruz...
vectorizer = CountVectorizer( max_features = 8500 )

# train verilerimizi feature vektöre matrisine çeviriyoruz
train_x = vectorizer.fit_transform(train_x)



In [521]:
train_x

<22500x8500 sparse matrix of type '<class 'numpy.int64'>'
	with 1924694 stored elements in Compressed Sparse Row format>

In [522]:
# array e dönüştürüyoruz
train_x = train_x.toarray()
train_y = y_train

In [523]:
train_x.shape, train_y.shape

((22500, 8500), (22500,))

In [524]:
train_y

array([1, 0, 0, ..., 1, 1, 0])

### Random Forest Modeli oluşturuyoruz ve fit ediyoruz

In [525]:
model = RandomForestClassifier(n_estimators = 460, random_state=55)
model.fit(train_x, train_y)


RandomForestClassifier(n_estimators=460, random_state=55)

### Test data kısmı için

In [526]:
# Test verilerimizi feature vektöre matrisine çeviriyoruz
# Yani aynı işlemleri(bag of wordse dönüştürme) tekrarlıyoruz bu sefer test datamız için:
test_xx = vectorizer.transform(test_x)


In [527]:
test_xx

<2500x8500 sparse matrix of type '<class 'numpy.int64'>'
	with 211253 stored elements in Compressed Sparse Row format>

In [528]:
test_xx = test_xx.toarray()

In [529]:
test_xx.shape

(2500, 8500)

#### Tahmin kısmı

In [530]:
testOfPrediction = model.predict(test_xx)
accuracy = roc_auc_score(y_test, testOfPrediction)

In [531]:
print("Auccuracy rate : % ", accuracy * 100)

Auccuracy rate : %  86.76359129829162
